In [1]:
import pandas as pd
import numpy as np
import time

import PeriodMaximisation as pm
from Battery import Battery

%run ./FirstModel.ipynb

In [2]:
def ConvertFormat_StoA(df):
    
    # Converting simple format to actual format
    df["Power"] = np.where(df["Status"] == "Charge", df["Actual"] * 2, df["Actual"] * 2 / 0.9)
    df = pm.renameColumns(df[["Time", "Price", "Power", "Actual", "Opening Capacity", "Closing Capacity"]])
    
    return df

def ConvertFormat_AtoS(act, df):
    
    # Converting actual format to simple format
    act = act[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)", 
               "Market Dispatch (MWh)", "Opening Capacity (MWh)", 
               "Closing Capacity (MWh)"]] # Select time and victoria prices
    act.columns = ["Time", "Price", "Actual", "Opening Capacity", "Closing Capacity"] # Rename columns
    #act["Time"] = pd.to_datetime(df["Time"]) # Convert data type
    #act = act.sort_values("Time").reset_index(drop = True) # Finalise
    act["Restrict"] = - np.where(act["Actual"] <= 0, act["Actual"] * 0.9, act["Actual"] / 0.9)
    act["Status"] = np.where(act["Actual"] < 0, "Charge", np.where(act["Actual"] > 0, "Discharge", "Do Nothing"))
    #act.index += 1
    #sim = pd.merge(df["Time"], act.drop("Time", axis = 1), left_index = True, right_index = True)
    # Questions:
    # why merge instead of just selecting and use the actual? and you don't need to convert it todatetime as it already datetime
    # when you read the file.
    
    return act

In [3]:
def model1(df, n = 17):
    Model = MovingAverage(n, df)
    Model = Maximisation(Model, df)
    Model = LossRemoval(Model)
    Model = Stationary(Model, df)
    Model = ShiftAction(Model, df)
    
    return Model

def model2(df):
    
    # Converting format
    df = ConvertFormat_StoA(df)
    
    # Get time period
    time = pm.getTimePeriod(df)
    
    # Run Dependendy Optimisation Algorithm
    findf = pm.DependencyOptimisation(df, time)
    
    # Convert back format
    findf = ConvertFormat_AtoS(findf, df)
    
    return findf

def model3(df):
    
    Model = Maximisation(df = df)
    Model = Stationary(Model)
    Model = ShiftAction(Model)

    return Model

def finalModel(original_data):
    tmp = original_data.copy()
    start = time.time()
    print("Calculating First Step")
    df = model1(tmp, n = 17)
    first = time.time()
    print(f"It took {first - start} seconds for First Step")
    print("Calculating Second Step")
    df = model2(df)
    second = time.time()
    print(f"It took {second - first} seconds for Second Step")
    print("Calculating Final Step")
    df = model3(df)
    third = time.time()
    print(f"It took {third - second} seconds for Third Step")
    print("\n")
    total = time.time()
    print(f"In total it took {total - start} seconds")
    return df

### All Data

In [ ]:
df = pd.read_excel("../../preprocessed_data/original_data.xlsx") # Open original data
df = df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
df.columns = ["Time", "Price"] # Rename columns
df["Time"] = pd.to_datetime(df["Time"]) # Convert data type
df = df.sort_values("Time").reset_index(drop = True) # Finalise

In [ ]:
final_result = finalModel(df)

### Training Set

In [ ]:
train_df = pd.read_excel("../../preprocessed_data/training_data.xlsx") # Open original data
train_df = train_df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
train_df.columns = ["Time", "Price"] # Rename columns
train_df["Time"] = pd.to_datetime(train_df["Time"]) # Convert data type
train_df = train_df.sort_values("Time").reset_index(drop = True) # Finalise

In [ ]:
train_df

### Testing Set

In [4]:
test_df = pd.read_excel("../../preprocessed_data/testing_data.xlsx") # Open original data
test_df = test_df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
test_df.columns = ["Time", "Price"] # Rename columns
test_df["Time"] = pd.to_datetime(test_df["Time"]) # Convert data type
test_df = test_df.sort_values("Time").reset_index(drop = True) # Finalise

In [5]:
test_df

,Time,Price
0,2021-07-01 00:30:00,90.51
1,2021-07-01 01:00:00,73.91
2,2021-07-01 01:30:00,33.79
3,2021-07-01 02:00:00,43.57
4,2021-07-01 02:30:00,45.90
...,...,...
2155,2021-08-14 22:00:00,49.93
2156,2021-08-14 22:30:00,62.86
2157,2021-08-14 23:00:00,32.26
2158,2021-08-14 23:30:00,25.10


In [6]:
final_testing_result = finalModel(test_df)

Calculating First Step
It took 4.111382484436035 seconds for First Step
Calculating Second Step
It took 33.51155948638916 seconds for Second Step
Calculating Final Step


TypeError: Maximisation() missing 1 required positional argument: 'MX'